# Luotuo Embedding 骆驼嵌入: Generative Text Embedding Model distilled from OpenAI API

骆驼嵌入是一个文本嵌入(text embedding)模型，由冷子昂, 刘思诣, 黄泓森, 陈舒年, 胡婧, 孙骜, 陈启源, 李鲁鲁等开发

<details>
  <summary> 每个作者都是第一作者，顺序是随机的。(点这里具体)</summary>

李鲁鲁发起了项目，并完成了初步的验证，提出了KL散度Loss和Hard Negative挖掘。

刘思诣完成了初步训练框架的编写，以及支撑了后面模型上传到hugging face管线。

冷子昂完成了完整的大模型和小模型的训练，包括载入数据和损失函数的实现。

陈启源准备了CNewSum的数据，做了句子切分。

黄泓森负责爬取了OpenAI Embedding的数据。

陈舒年完成了重要的几个可视化。

孙骜（即将）用我们的得到的Embedding，完成CoT的提升实验。

胡婧收集了周杰伦的歌词，并（即将）完成更多的定量实验。

</details>

骆驼嵌入是[Luotuo(骆驼)](https://github.com/LC1332/Luotuo-Chinese-LLM)的子项目之一, 后者由李鲁鲁, 冷子昂, 陈启源发起。


## 安装环境

In [ ]:
# Requirements
!pip install transformers
!pip install openai
!pip install openTSNE
!pip install datasets

## 从骆驼嵌入项目中获取数据

## 从Hugging Face中获取模型

我们的骆驼BERT的主要模型为luotuo-bert

现在暂时是110M的小模型。之后中模型上线之后,luotuo-bert会替换为300M以上的较大的模型，同时小模型会更名为luotuo-bert-small

In [1]:
import torch
from scipy.spatial.distance import cosine
from transformers import AutoModel, AutoTokenizer
from argparse import Namespace
# Import our models. The package will take care of downloading the models automatically
tokenizer = AutoTokenizer.from_pretrained("silk-road/luotuo-bert")
model_args = Namespace(do_mlm=None, pooler_type="cls", temp=0.05, mlp_only_train=False, init_embeddings_model=None)
model = AutoModel.from_pretrained("silk-road/luotuo-bert", trust_remote_code=True, model_args=model_args)

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


## t-SNE 文本可视化实验

In [12]:
import pandas as pd
tsne_data_label_path = "../data/sentpair_label.csv"

data = pd.read_csv(tsne_data_label_path, header=0)
text_left = data["Column1"].tolist()
text_right = data["Column2"].tolist()
label = data["Label"].tolist()

In [4]:
import csv
import numpy as np
import sys
sys.path.append("..")


inputs = tokenizer(text_left, padding=True, truncation=True, return_tensors="pt")
with torch.no_grad():
    embeddings_left = model(**inputs, output_hidden_states=True, return_dict=True, sent_emb=True).pooler_output
inputs = tokenizer(text_right, padding=True, truncation=True, return_tensors="pt")
with torch.no_grad():
    embeddings_right = model(**inputs, output_hidden_states=True, return_dict=True, sent_emb=True).pooler_output
    
cos_sim_matrix = torch.matmul(embeddings_left, embeddings_right.t())
cos_sim_matrix /= torch.matmul(torch.norm(embeddings_left, dim=1, keepdim=True), torch.norm(embeddings_right, dim=1, keepdim=True).t())
tensor_cpu = cos_sim_matrix.cpu()

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


### T-SNE with labels

In [5]:
from lib.tsne import TSNE_Plot

merged_list = text_left + text_right
label = label + label
merged_embed = torch.cat((embeddings_left, embeddings_right), dim=0)

# if the data have no labels, you can use the following code to cluster the data
tsne_plot = TSNE_Plot(merged_list, merged_embed, label = label)
tsne_plot.tsne_plot(n_sentence=40)

Perplexity value 30 is too high. Using perplexity 12.33 instead
Perplexity value 30 is too high. Using perplexity 12.33 instead


### T-SNE without labels
当语料label没有给定时，算法将自动聚类并绘制T-SNE plot

In [8]:
import ast
tsne_data_cluster_path = "../data/search_data.csv"
tsne_data_cluster = pd.read_csv(tsne_data_cluster_path, header=0)
text = tsne_data_cluster["sentence"].tolist()
embed = tsne_data_cluster["embed"].apply(lambda x: ast.literal_eval(x))

tsne_plot_cluster = TSNE_Plot(text, embed, n_clusters=4)
tsne_plot_cluster.tsne_plot(n_sentence=40)

## 热力图

In [11]:
from lib.heatmap import Heatmap
heat_plot_data_path = "../data/sentspair_embed.csv"
heat_plot_data = pd.read_csv(heat_plot_data_path, header=0)
first = heat_plot_data["first"].tolist()
second = heat_plot_data["second"].tolist()
first_embed = heat_plot_data["first_embed"].tolist()
second_embed = heat_plot_data["second_embed"].tolist()
# positions = [(i, i) for i in range(0, 20, 2)] + [(1, 5), (2, 3), (15, 9), (5, 13), (17, 7)]
# heatmap = Heatmap(df, positions)
df = pd.DataFrame({ "first":text_left, 
                    "second":text_right, 
                    "first_embed":[np.array(embeddings_left[i]) for i in range(len(embeddings_left))], 
                    "second_embed":[np.array(embeddings_right[i]) for i in range(len(embeddings_right))]})
heatmap = Heatmap(df)
heatmap.create_heatmap(font_path='./lib/arial.ttf')

## 模糊搜索

In [43]:
import ast
import lib.fuzzySearch
import importlib

importlib.reload(lib.fuzzySearch)
from lib.fuzzySearch import FuzzySearch
fuzzy_search_data_path = "../data/search_data.csv"
stop_words_path = "../data/stop_word.txt"

fuzzy_search_data = pd.read_csv(fuzzy_search_data_path, header=0)
fuzzy_search_data["embed"] = fuzzy_search_data["embed"].apply(lambda x: ast.literal_eval(x))

In [44]:
# fuzzy_search_data is the data source to search from, which is a pandas dataframe and should have two columns: sentence and embed
# sentence is the content of the sentence, each row is a string
# embed is the sentence embedding, each row is a list of float
fuzzy_search = FuzzySearch(fuzzy_search_data, stop_words_path = stop_words_path)

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


In [45]:
#虎扑报道马刺的保罗-加索尔与球队正式签订协议，有哪些相关的新闻？
fuzzy_search.search()

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


第1个结果：
<mark>虎扑</mark>体育7月14日讯。<mark>马刺</mark>官网宣布,<mark>保罗</mark>-加<mark>索尔</mark>与<mark>球队正式签订协议</mark>。根据之前的<mark>报道</mark>,加<mark>索尔</mark>与<mark>马刺签订</mark>的是一份为期2年3000万的<mark>协议</mark>。加<mark>索尔</mark>上赛季共打了72场比赛,场均31.8分钟,贡献16.5分11.0篮板4.1助攻2.0盖帽。

第2个结果：
2015-09-2809:03。新浪体育。显示图片下一站丹佛?北京时间9月28日,据雅<mark>虎</mark>体育<mark>报道</mark>,波特兰开拓者队已经与老将射手迈克-米勒达成了合同买断。一旦米勒成为自由球员之后,丹佛掘金队有意得到他。据雅<mark>虎</mark>体育得到的消息,如果迈克-米勒在规定的时间内没有被<mark>球队</mark>认领并成为一名自由球员,联盟里有一些<mark>球队</mark>将会对他感兴趣,其中就包括掘金队。目前,掘金队中已经有14个保障合同。事实上早在一年前,掘金队就在自由球员市场上猛追过米勒。但最终,米勒决定跟随勒布朗-詹姆斯加盟骑士队。今年7月,开拓者队在关于布兰登-海伍德的那笔交易中得到了迈克-米勒。作为2000年的NBA首轮5号新秀,迈克-米勒曾获得过2001年最佳新秀称号、2006年最佳第六人称号,并且曾跟随迈阿密热火队夺得过两次NBA总冠军。在长达15年的职业生涯里,米勒一直被视为一名关键时刻靠得住的射手,并且能够为争冠<mark>球队</mark>做出贡献。他的生涯场均数据是11.3分、4.4个篮板和2.7次助攻。

第3个结果：
<mark>虎扑</mark>体育9月3日讯。根据《太阳哨兵报》记者IraWinderman的<mark>报道</mark>,帕特-莱利本周接受采访时表示,交易得到沙奎尔-奥尼尔是他们队史上最大的交易收获。在2004年夏天,奥尼尔被湖人交易到了热火,随后他就与德维恩-韦德在2005-06赛季一起为热火拿到了他们队史上的首个总冠军。“我会这么说,我的意思也是这个,”莱利说,“获得奥尼尔比我们史上的任何收获都重要,其中也包括三巨头。”莱利称热火交易得到奥尼尔的重要

# TODO:
* 模糊问题搜索
* 文本聚类
* 少样本分类学习